In [241]:
import numpy as np
import scipy.stats as ss
np.set_printoptions(precision=2, suppress=True)
rng = np.random.default_rng(1)
np.random.seed(1)

GBM :

$dS_t = S_t \mu dt + S_t \sigma W_t, \qquad W_t \sim N(0, dt)$

$S_T = S_0 \times e^{(\mu - \frac{\sigma^2}{2}) T + \sigma \sqrt{T} Z} $

In [242]:
def GBM(S0, mu, sigma, T):
    np.random.seed(1)
    # rng = np.random.default_rng(1)
    Z = np.random.standard_normal(1)
    S_T = S0 * np.exp((mu - sigma**2 / 2)*T + sigma * np.sqrt(T) * Z)
    return S_T

S0 = 100
mu = 0.1
sigma = 0.2
T = 1

GBM(S0, mu, sigma, T)

array([149.91])

In [243]:
def BS_price(S0, K, r, sigma, T, t, call_or_put="C"):
    T = T-t
    d1 = (np.log(S0/K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    print("d1 =", d1)
    d2 = d1 - sigma * np.sqrt(T)
    
    if call_or_put == "C":
        return S0 * ss.norm.cdf(d1) - np.exp(-r*T) * K * ss.norm.cdf(d2)
    else:
        return np.exp(-r*T) * K * ss.norm.cdf(-d2) - S0 * ss.norm.cdf(-d1)

In [244]:
BS_price(12.5, 15, 0.01, 0.45, 1, 0, "P")

d1 = -0.15793679287545465


3.7827324909512487

In [245]:
d1 = -0.15793679287545465
N_d1 = ss.norm.cdf(-0.15793679287545465)
N_d1 - 1

-0.5627466973133798

In [246]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import lognorm, norm

# Write a Black-Scholes pricer function
def BS(S0, r, sigma, t, T, K, call_or_put="C"):
    dt = T - t
    d1 = (np.log(S0/K) + (r + 1/2*sigma**2)*dt) / (sigma*np.sqrt(dt))
    d2 = (np.log(S0/K) + (r - 1/2*sigma**2)*dt) / (sigma*np.sqrt(dt))
    if call_or_put == "C":  # if call
        return S0 * norm.cdf(d1) - np.exp(-r*dt) * K * norm.cdf(d2)
    else:  # if put
        return np.exp(-r*dt) * K * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    
def MC_price(S0, K, r, sigma, T, M, Ite, t, call_or_put="C"):
    np.random.seed(42)
    T = T - t
    dt = T / M
    S = np.zeros((M+1, Ite))
    S[0] = S0  # at t=0, all prices are S0. (first row is S0 repeated.)
    rn = np.random.standard_normal((M, Ite))
    multipliers = np.exp((r - sigma**2 / 2) * dt + sigma * np.sqrt(dt) * rn)
    print("multipliers.cumprod() =\n", multipliers.cumprod(axis=0))
    S[1:, :] = S0 * multipliers.cumprod(axis=0)  # price paths are generated
    print("S =\n", S)
    
    if call_or_put == "C":
        average_payoff = np.maximum(S[-1, :] - K, 0).mean()
    else:
        average_payoff = np.maximum(K - S[-1, :], 0).mean()
    
    opt_price = np.exp(-r*dt) * average_payoff
    return opt_price

In [247]:
K = 23
S0 = 22.75
r = 0.02
T = 7/12
sigma = 0.45

BS_price(S0, K, r, sigma, T, 0, "C")

d1 = 0.1739926775249994


3.1113021537768724

In [248]:
MC_price(S0, K, r, sigma, T, 2500, 2500, 0, "C")

multipliers.cumprod() =
 [[1.   1.   1.   ... 1.   1.01 1.  ]
 [1.01 1.01 1.   ... 1.02 1.01 1.  ]
 [1.   1.   0.99 ... 1.03 0.99 1.  ]
 ...
 [1.06 1.23 1.24 ... 1.02 1.01 1.4 ]
 [1.06 1.23 1.23 ... 1.02 1.01 1.41]
 [1.06 1.23 1.23 ... 1.03 1.01 1.42]]
S =
 [[22.75 22.75 22.75 ... 22.75 22.75 22.75]
 [22.83 22.73 22.85 ... 22.71 22.87 22.8 ]
 [22.92 22.88 22.8  ... 23.2  23.   22.67]
 ...
 [24.08 27.97 28.12 ... 23.2  22.94 31.76]
 [24.19 28.06 27.89 ... 23.13 22.87 32.03]
 [24.15 27.99 28.01 ... 23.35 22.9  32.33]]


3.175239308600649

In [249]:
K = 124
S0 = 124
r = 0.0175
T = 3/12
sigma = 0.25

BS_price(S0, K, r, sigma, T, 0, "C")

d1 = 0.0975


6.440522540754287

In [250]:
def vasicek(r0, k, theta, sigma, T, N, M):
    """_summary_

    Args:
        r0 (_type_): starting interest rate
        k (_type_): speed of mean-reversion
        theta (_type_): long term average of interest rates
        sigma (_type_): short-term volatility of interest rates
        T (_type_): maturity
        N (_type_): num of steps
        M (_type_): num of paths
    """
    np.random.seed(42)
    dt = T / N
    rn = np.random.standard_normal((N, M))
    R = np.zeros((N, M))
    R[0, :] = r0  # first row is initial rates
    print("R =\n", R)
    for i in range(N-1):
        # print("R[i, :] =\n", R[i, :])
        dr = k * (theta - R[i, :]) * dt + sigma * np.sqrt(dt) * rn[i, :]
        R[i+1, :] = R[i, :] + dr
    return R

M = 1  # Number of paths for MC
N = 150  # Number of steps
T = 1.0  # Maturity
r0 = 0.01875
k = 0.25
theta = 0.02
sigma = 0.0180

rates = vasicek(r0, k, theta, sigma, T, N, M)
rates_min = rates[-1, :].min()
rates_max = rates[-1, :].max()

print("rates_mean =", rates[-1].mean())
print("rates_min =", rates_min)
print("rates_max =", rates_max)

R =
 [[0.02]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]
 [0

In [251]:
def vasicek(r0, K, theta, sigma, T, N, M):
    np.random.seed(42)
    dt = T / N
    rates = np.zeros((N, M))
    rates[0, :] = r0
    for j in range(M):
        for i in range(1, N):
            dr = (
                K * (theta - rates[i - 1, j]) * dt
                + sigma * np.sqrt(dt) * np.random.normal()
            )
            rates[i, j] = rates[i - 1, j] + dr
    return rates

rates = vasicek(r0, k, theta, sigma, T, N, M)
rates[-1,  :].mean()

0.0030317874529839513